In [21]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as pl

In [6]:
data= pd.read_csv('C:/Users/HP/Documents/challenge sécurité/challenge-2023/données/log_fw_3.csv', sep=';', header=None)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  object 
 1   1       999998 non-null   object 
 2   2       999998 non-null   object 
 3   3       999998 non-null   object 
 4   4       999998 non-null   float64
 5   5       999998 non-null   float64
 6   6       999998 non-null   float64
 7   7       999998 non-null   object 
 8   8       999998 non-null   object 
 9   9       0 non-null        float64
 10  10      999998 non-null   float64
dtypes: float64(5), object(6)
memory usage: 83.9+ MB


In [8]:
def to_datetime(x):
    return pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S.%f')

def prepare_log_data(log_data):
    log_data.columns = ['timestamp', 'IPsrc', 'IPdst', 'proto', 'portsrc', 'portdst', 'rule','action','7', '8','9']
    log_data.timestamp = to_datetime(log_data.timestamp)
    log_data.portdst = log_data.portdst.astype('object')
    return log_data

In [9]:
data = prepare_log_data(data)

In [10]:
data.pop('8')

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
999995   NaN
999996   NaN
999997   NaN
999998   NaN
999999   NaN
Name: 8, Length: 1000000, dtype: float64

In [11]:
data.head(10)

,timestamp,IPsrc,IPdst,proto,portsrc,portdst,rule,action,7,9
0,2023-02-12 03:59:03,66.249.69.180,17.17.17.17,TCP,57630.0,443.0,1.0,PERMIT,eth0,6.0
1,2023-02-12 03:59:04,66.249.69.178,17.17.17.17,TCP,63808.0,443.0,1.0,PERMIT,eth0,6.0
2,2023-02-12 03:59:08,221.11.125.141,17.17.17.17,TCP,43123.0,2376.0,999.0,DENY,eth0,6.0
3,2023-02-12 03:59:18,183.136.225.42,17.17.17.17,TCP,8088.0,70.0,999.0,DENY,eth0,6.0
4,2023-02-12 03:59:20,38.32.112.34,17.17.17.17,TCP,6356.0,443.0,1.0,PERMIT,eth0,6.0
5,2023-02-12 03:59:22,45.125.66.18,17.17.17.17,TCP,58438.0,23.0,7.0,DENY,eth0,6.0
6,2023-02-12 03:59:29,107.170.242.5,17.17.17.17,TCP,39974.0,5093.0,999.0,DENY,eth0,6.0
7,2023-02-12 03:59:33,45.93.16.171,17.17.17.17,TCP,42021.0,50802.0,999.0,DENY,eth0,6.0
8,2023-02-12 03:59:35,157.55.39.59,17.17.17.17,TCP,45120.0,443.0,1.0,PERMIT,eth0,6.0
9,2023-02-12 03:59:39,104.156.155.11,17.17.17.17,TCP,58728.0,9109.0,999.0,DENY,eth0,6.0


In [12]:
data.portsrc = data.portsrc.astype('object')

In [13]:
data.dtypes

timestamp    datetime64[ns]
IPsrc                object
IPdst                object
proto                object
portsrc              object
portdst              object
rule                float64
action               object
7                    object
9                   float64
dtype: object

In [14]:
data.dropna(inplace=True)

In [15]:
data.head()

,timestamp,IPsrc,IPdst,proto,portsrc,portdst,rule,action,7,9
0,2023-02-12 03:59:03,66.249.69.180,17.17.17.17,TCP,57630.0,443.0,1.0,PERMIT,eth0,6.0
1,2023-02-12 03:59:04,66.249.69.178,17.17.17.17,TCP,63808.0,443.0,1.0,PERMIT,eth0,6.0
2,2023-02-12 03:59:08,221.11.125.141,17.17.17.17,TCP,43123.0,2376.0,999.0,DENY,eth0,6.0
3,2023-02-12 03:59:18,183.136.225.42,17.17.17.17,TCP,8088.0,70.0,999.0,DENY,eth0,6.0
4,2023-02-12 03:59:20,38.32.112.34,17.17.17.17,TCP,6356.0,443.0,1.0,PERMIT,eth0,6.0


In [19]:
# get IPsrc, IPdst, portsrc, portdst, action, rule, 9 into a new dataframe
X = data[['portsrc', 'portdst', 'rule', '9']]
print(X.head())

print('\n')
y = data['action']
print(y.value_counts(normalize=True))

   portsrc portdst   rule    9
0  57630.0   443.0    1.0  6.0
1  63808.0   443.0    1.0  6.0
2  43123.0  2376.0  999.0  6.0
3   8088.0    70.0  999.0  6.0
4   6356.0   443.0    1.0  6.0


PERMIT    0.764711
DENY      0.235289
Name: action, dtype: float64


In [30]:
# get IPsrc, IPdst, portsrc, portdst, action, rule, 9 into a new dataframe  X
data = data[['IPsrc', 'IPdst','portsrc', 'portdst', '9']]

# recode IPsrc and IPdst to numeric values
data['IPsrc'] = data['IPsrc'].astype('category')
data['IPdst'] = data['IPdst'].astype('category')

data['IPsrc'] = data['IPsrc'].cat.codes
data['IPdst'] = data['IPdst'].cat.codes

print(data.head())


   IPsrc  IPdst  portsrc portdst    9
0  27186      0  57630.0   443.0  6.0
1  27184      0  63808.0   443.0  6.0
2  20538      0  43123.0  2376.0  6.0
3  14567      0   8088.0    70.0  6.0
4  22895      0   6356.0   443.0  6.0


In [18]:
# Train test split
from sklearn.model_selection import train_test_split
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [26]:
#supprimer la colonne date
data = data.drop(["timestamp"], axis=1)

In [ ]:
# Specify hyperparameter search space
param_grid = {
    "n_clusters": [2, 3, 4, 5, 6],
    "init": ["k-means++", "random"],
    "n_init": [10, 20, 30],
    "max_iter": [100, 200, 300],
}

# Define KMeans estimator
kmeans = KMeans()

# Define GridSearchCV object
grid_search = GridSearchCV(
    kmeans, param_grid=param_grid, scoring="silhouette", n_jobs=-1
)

# Fit GridSearchCV object to data
grid_search.fit(data)

# Print best parameters and score
print(
    f"Best parameters: {grid_search.best_params_}, Best silhouette score: {grid_search.best_score_}"
)

# Get best estimator
best_estimator = grid_search.best_estimator_

# Use best estimator for clustering
clusters = best_estimator.fit_predict(data)


In [ ]:
# Visualiser les clusters
data["Cluster"] = kmeans.labels_
sns.scatterplot(x="Port", y="Destination IP", hue="Cluster", data=data)
plt.show()

In [ ]:
# Déterminer le nombre optimal de clusters en utilisant la méthode de Silhouette
scores = []
k_values = range(2, 10)
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    score = silhouette_score(X, kmeans.labels_)
    scores.append(score)

# Tracer le graphique de la silhouette
sns.lineplot(x=k_values, y=scores)
plt.xlabel("Nombre de clusters")
plt.ylabel("Score de silhouette")
plt.title("Recherche du nombre optimal de clusters")
plt.show()

# Utiliser le nombre optimal de clusters pour le clustering final
k = np.argmax(scores) + 2
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)

# Ajouter les clusters au jeu de données d'origine
data["Cluster"] = kmeans.labels_

# Tracer le graphique de dispersion des clusters
sns.scatterplot(x="Destination Port", y="Source Port", hue="Cluster", data=data)
plt.xlabel("Port de destination")
plt.ylabel("Port source")
plt.title("Clustering des adresses IP et des ports")
plt.show()